<a href="https://colab.research.google.com/github/vaibhawvipul/Reinforcement-Learning-Lectures/blob/master/Introduction_to_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Tensorflow

Developed by Google Brain . Written in Python, c++ and CUDA.

![Tensorflow Logo](https://www.tensorflow.org/_static/images/tensorflow/logo.png)


---

Tensorflow framework is composed of two things - 

1.   A library for defining computational graphs
2.   A runtime for executing the graphs on different hardwares

## What are computational Graphs?

Computational graphs are an abstract way to represent or describe computation as directed graph:

*   Edges are **tensors** or multi-dimensional arrays
*   Nodes are rules or **Ops** which manipulates the tensors.

Let us understand the computational graphs a bit more - 

Suppose we have to execute following operations - 

```
a = 10

b = 20 

temp1 = a + b

temp2 = a*b

result = temp2/temp1

print result
```

Now for the mentioned flow, this is how a computational graph will look like - 

[Click here to see the computational graph](https://github.com/vaibhawvipul/Reinforcement-Learning-Lectures/blob/master/TF-Comp-graph.png)

This representation helps tensorflow to compute dependencies in the graph and allows the tensorflow backend to parallelize the processes on multiple cores on your system.
It gives very high performance with no inputs from user about the code execution.

Now let us see how can we define the graph shown above in tensorflow - 


In [0]:
import tensorflow as tf
tf.__version__


In [0]:
# defining a and b
a = tf.constant(10)
b = tf.constant(20)

# defining some more ops
temp1 = tf.add(a,b)
temp2 = tf.multiply(a,b)

result = tf.divide(temp2,temp1)

print(result)

Tensorflow supports various forms of computation - stateful, conditional, iterative, async.

All the above forms of computations are supported because of variety of **ops** tensorflow provides.


*   **Variable** - to read and update the writable values that persists across execution.
*   **Conditional**
*   **Loop**
*   **Control**
*   **Queue**

---

## Definition vs Execution

A tensorflow program consists of two parts - 

*   Definition
*   Execution

`tf.Graph` is used to specify computations. `tf.Session` is responsible for executing sessions.


---

## tf.Graph

It is used to descibe computation using Ops and Tensors.

A tensor may have shape and data type but no actual data.


In [0]:
#1000000000000x1000000000000
a = tf.zeros((int(1e12), int(1e12))) 

In [0]:
# following won't work, It will give OOM error
import numpy as np
np.zeros((int(1e12), int(1e12)))

Tensor shapes can be derived from the graph(shape inference)

In [0]:
a = tf.zeros((10,10)) #10X10 shape

b = tf.concat([a,a],axis=0)
b.shape

## tf.Session

This actually executes the graph. It does the computation.

In [0]:
a = tf.constant(1.0)
b = tf.constant(1.0)
c = tf.constant(4.0)

d = tf.divide(c,tf.add(a,b))

with tf.Session() as sess:
  print(sess.run(d))

## Execution model

`sess.run()` identifies and executes the smallest set of nodes required to compute the requested tensor.

## Programming Languages - 


1.   Python
2.   C++
3.   Rust
4.   GO
5.   Java

No matter which language you choose, most of the computation will happen on the highly optimized tensorflow C++ backend.

## Variable Initializer - 

One can initialize all the vaiable at once like 


In [0]:
init = tf.global_variables_initializer()

## Placeholders and Feeds 
 
Placeholders are used to insert data in the graph at runtime.



In [0]:
x = tf.compat.v1.placeholder(tf.float32, shape=(1024, 1024))
y = tf.matmul(x, x)

with tf.compat.v1.Session() as sess:
  #print(sess.run(y))  # ERROR: will fail because x was not fed.

  rand_array = np.random.rand(1024, 1024)
  print(sess.run(y, feed_dict={x: rand_array}))  # Will succeed.

Notes - 

1. Using placeholder is manully intensive
2. Gives us a lor of flexibility
3. tf.data is better option

# Writing first neural network in keras with tf backend





In [0]:
import tensorflow as tf
import numpy
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical

# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#print len(X_train)

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# flatten 28*28 images to a 784 vector for each image
num_pixels = X_train.shape[1] * X_train.shape[2] #m X n array where m = n = 28
#print num_pixels #print 784
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
#print X_train
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]
#print y_train

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
	# Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))